In [1]:
%%time
import pandas as pd
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
#from numpy import loadtxt
from keras.utils import np_utils
#from sklearn.preprocessing import LabelEncoder

from keras.callbacks import EarlyStopping

from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize

Wall time: 4.22 s


In [2]:
%%time
df = pd.read_excel("BQ Analysis v1_1.xlsx", "Project BQ")

Wall time: 10.1 s


In [3]:
%%time
df = df.astype(str)
df["PREPROCESS_TEXT"] = df["BQ_TRADE"] + "," + df["BQ_HEADING"] + "," + df["BQ_SUB_HEADING"] + "," + df["BQ_ITEM_HEADING"] + "," + df["BQ_ITEM_DESCRIPTION"] #+ "," + df["AC_TRADE_CODE"]
#df['RESULT'] = ""
#training_df = df.loc[df['CONTRACT_CODE'] == "FL49"].copy() // not enough training data?
df = df[df['AC_TRADE_CODE'] != "DA"]
df = df[df['AC_TRADE_CODE'] != "IP"]
df = df[df['AC_TRADE_CODE'] != "SS"]
df = df[df['AC_TRADE_CODE'] != "EL"]
df = df[df['AC_TRADE_CODE'] != "SO"]
df = df[df['AC_TRADE_CODE'] != "AS"]


df["PREPROCESS_TEXT"] = list(map(lambda x: x.lower(), list(df["PREPROCESS_TEXT"])))





Wall time: 353 ms


In [4]:
class_names = ['PS',
'ST',
'SM',
'EL',
'PL',
'BS',
'DR',
'CJ',
'PA',
'AR',
'PR',
'SO',
'GL',
'IF',
'DW',
'EX',
'IR',
'FD',
'AP',
'SU',
'FF',
'SG',
'AS',
'EF',
'BR',
'SS',
'DA',
'IP'
]





In [5]:
training_df = df[df['CONTRACT_CODE'] != "QHP3"].copy()


training_df["AC_TRADE_CODE_INDEX"] = list(map(lambda x: class_names.index(x), list(training_df.AC_TRADE_CODE)))
#training_df["AC_TRADE_CODE"].value_counts()
#training_df["AC_TRADE_CODE_INDEX"].value_counts()

In [6]:
test_df = df[df['CONTRACT_CODE'] == "QHP3"].copy()
test_df["AC_TRADE_CODE_INDEX"] = list(map(lambda x: class_names.index(x), list(test_df.AC_TRADE_CODE)))
test_df["AC_TRADE_CODE_INDEX"].value_counts()

0     2788
1     2712
2     2258
4      853
6      479
8      427
7      368
9      215
12      95
10      58
24      35
16      24
15      10
Name: AC_TRADE_CODE_INDEX, dtype: int64

In [7]:
# print(len(df["AC_TRADE_CODE"]))
# label_encoder = LabelEncoder()
# label_encoder.fit(df["AC_TRADE_CODE"])
# training_label = np_utils.to_categorical((label_encoder.transform(df["AC_TRADE_CODE"])))

# print(len(training_label))

In [8]:
# %%time
# tokenized_doc = []
# for d in training_df["PREPROCESS_TEXT"]:
#     tokenized_doc.append(word_tokenize(d))
# tokenized_doc
# tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_doc)]
# len(tagged_data)

In [9]:
%%time
model= Doc2Vec.load("test_doc2vec.model")
#training_text = model.wv.vectors
#print(len(training_text))

Wall time: 247 ms


In [10]:
%%time
# test_data = word_tokenize(df["PREPROCESS_TEXT"])
# v1 = model.infer_vector(test_data)
# print("V1_infer", v1)

training_text = []
for index, row in training_df.iterrows():
    training_text.append(model.infer_vector(word_tokenize(row["PREPROCESS_TEXT"])))

training_text = np.asarray(training_text)  
print(len(training_text))
print(training_text[0])





test_text = []
for index, row in test_df.iterrows():
    #print(model.infer_vector(word_tokenize(row["PREPROCESS_TEXT"])))
    test_text.append(model.infer_vector(word_tokenize(row["PREPROCESS_TEXT"])))

test_text = np.asarray(test_text)  
print(len(test_text))
print(test_text[0])


31075
[-0.06537517 -0.09960778 -0.42488965 -0.6178617  -0.9215546   0.27969044
 -0.53391707 -0.909513   -1.4888742  -0.6120924  -0.46929762  0.32160908
 -0.10723372 -1.4194572   0.6695041   0.896733   -0.16387694  0.789303
  0.82672787  1.0911089 ]
10322
[ 4.89012420e-01  1.28465796e+00 -1.17283487e+00  5.57047546e-01
  4.89087433e-01 -1.47226942e+00 -1.04352735e-01 -1.04560982e-03
 -3.86845171e-01 -1.66518539e-01  9.99648511e-01  6.59418464e-01
  1.74135447e+00  1.31072268e-01  3.76470447e-01 -6.16819620e-01
 -7.10797906e-01  1.03964150e+00 -3.04088086e-01  2.30819941e+00]
Wall time: 6min


In [11]:
training_label = np.asarray(training_df["AC_TRADE_CODE_INDEX"])  
test_label = np.asarray(test_df["AC_TRADE_CODE_INDEX"])  

In [12]:
training_text.shape, training_label.shape


((31075, 20), (31075,))

In [13]:

%%time


early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=2)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, )),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(50)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(training_text, training_label, epochs=20, callbacks=[early_stopping])


Epoch 1/20
972/972 [==============================] - 1s 803us/step - loss: 1.4353 - accuracy: 0.6316
Epoch 2/20
972/972 [==============================] - 1s 804us/step - loss: 0.5830 - accuracy: 0.8236
Epoch 3/20
972/972 [==============================] - 1s 803us/step - loss: 0.5033 - accuracy: 0.8444
Epoch 4/20
972/972 [==============================] - 1s 817us/step - loss: 0.4505 - accuracy: 0.86371s - l
Epoch 5/20
972/972 [==============================] - 1s 791us/step - loss: 0.4178 - accuracy: 0.8734
Epoch 6/20
972/972 [==============================] - 1s 774us/step - loss: 0.3949 - accuracy: 0.8800
Epoch 7/20
972/972 [==============================] - 1s 842us/step - loss: 0.3655 - accuracy: 0.8887
Epoch 8/20
972/972 [==============================] - 1s 829us/step - loss: 0.3416 - accuracy: 0.8964
Epoch 9/20
972/972 [==============================] - 1s 781us/step - loss: 0.3191 - accuracy: 0.9035
Epoch 10/20
972/972 [==============================] - 1s 781us/step - loss:

In [14]:
test_loss, test_acc = model.evaluate(test_text,  test_label, verbose=2)

print('\nTest accuracy:', test_acc)

323/323 - 0s - loss: 0.3269 - accuracy: 0.9133

Test accuracy: 0.9132919907569885


In [15]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_text)

In [16]:

i = 0
test_df["PREDICT_RESULT"] = 0
for index, row in test_df.iterrows():
    test_df["PREDICT_RESULT"][index] = np.argmax(predictions[i])
    i = i + 1


test_df["COMPARE"] = np.where(test_df["AC_TRADE_CODE_INDEX"] == test_df["PREDICT_RESULT"], True, False)


<ipython-input-16-7c42d552e199>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["PREDICT_RESULT"][index] = np.argmax(predictions[i])


In [17]:
test_df['COMPARE'].value_counts()

True     9427
False     895
Name: COMPARE, dtype: int64